# Develop, Train, Deploy Scikit-Learn SVC



#### [reference]    
https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker-python-sdk/scikit_learn_randomforest/Sklearn_on_SageMaker_end2end.ipynb


https://www.youtube.com/watch?v=hLzEHsUSHq4

In [ ]:
!pip install --upgrade pandas

In [ ]:
import sklearn # sklearn 버전확인 sklearn = scikit-learn
sklearn.__version__

## [Import Lib]

In [ ]:
# pip install --upgrade pip

In [7]:
# !pip install pandas

In [8]:
import pandas as pd
pd.__version__


'2.1.3'

In [9]:
import datetime                 #날짜 및 시간 관련 모듈
import time                     #시간관련 모듈
import tarfile                  #파일 압축 및 해제를 위한 모듈

import boto3                    #AWS SD
import pandas as pd
import numpy as np
from sagemaker import get_execution_role                #SageMaker 역할(실행역할을 가져오는 메서드)
import sagemaker                                        #SageMaker lib
from sklearn.model_selection import train_test_split    # train,test set 분리


sm_boto3 = boto3.client("sagemaker")                #SageMaker 클라이언트를 생성합니다. 

sess = sagemaker.Session()                          #SageMaker 세션을 생성합니다. SageMaker와 상호 작용할 때 사용

region = sess.boto_session.region_name              #현재 SageMaker 세션의 리전을 가져와서 region 변수에 저장

bucket = sess.default_bucket()  # this could also be a hard-coded bucket name

print("Using bucket " + bucket)                     #SageMaker 버킷의 이름을 출력 버킷(aws s3)(파일 저장하는 곳)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
Using bucket sagemaker-ap-northeast-2-918934048065


## Prepare data
We load a dataset from sklearn, split it and send it to S3

In [10]:
# df = pd.read_json('sqli_query_clean.json')
# df = pd.read_csv('security_log_clean.csv',encoding='utf-8')
# df = pd.read_csv('security_log_clean.csv')
df= pd.read_csv('security_log_complete.csv')


#### 결측치 제거

In [11]:
# 필요없는 열 제거
df.drop('Unnamed: 0', axis =1, inplace =True)
# axis =1 : 열  axis =0 : 행 즉, 열을 drop 없애버린다는 이야기

# inplace =True 하면 데이터프레임에 바로 적용
# df = df.drop('Unnamed: 0', axis =1) 이렇게 안해도 되는거

In [12]:
df.dropna(axis=0, inplace=True)

In [13]:
df.columns #열(특징) 확인

Index(['pattern', 'label'], dtype='object')

In [14]:
df.head() # 데이터프레임 확인

,pattern,label
0,3930aab9d3a38dcf,urlregex
1,2faf2430c9a04a39,urlregex
2,15953497afbbaf03,urlregex
3,/d/msdownload/update/others/2023/08/39551369_5...,urlregex
4,/c/msdownload/update/others/2023/08/39551110_5...,urlregex


In [15]:
df['label'].value_counts() # 어떤 라벨들이 있는지 확인

label
valid            12112
urlregex          2006
accesscontrol     1154
shellcode          754
upload             540
sqli               292
webattack          253
xss                218
includei            60
download            57
logical_sqli        54
Name: count, dtype: int64

#### train set, test set split 분리

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    df['pattern'], df['label'], test_size=0.2, random_state=42
)

trainX = pd.DataFrame(X_train)
trainX['label'] = y_train

testX = pd.DataFrame(X_test)
testX['label'] = y_test

In [17]:
trainX.head() # 데이터가 제대로 나눠져 있는지 확인

,pattern,label
8762,d=103&p=hysyfm1wqf bf53ag7l7npy0whwo41skgl3nmw...,valid
7540,p1=1691586991&p2=404&p3=2&p4=wzuaexh64bu/rhhm2...,valid
12272,c=13&p=1uhcd0lcweyfy1ijzvajh72l38d_9k4e0bgef_d...,valid
10710,p1=1691589321&p2=404&p3=2&p4=g rfnhkuec7x0zfqq...,valid
3646,"param=<!entity xxe system ""file:///etc/passwd"">",shellcode


In [18]:
type(X_train)

pandas.core.series.Series

trainx, testx 데이터 나눠놓은거 to_csv 해서 csv 파일 생성

In [19]:

trainX.to_csv("attack_train.csv")

testX.to_csv("attack_test.csv")

위에서 만들어진 csv 파일 s3 버킷(파일 저장하는 곳)에다가 업로드

In [20]:
# send data to S3. SageMaker will take training data from s3
trainpath = sess.upload_data(
    path="attack_train.csv", bucket=bucket, key_prefix="sagemaker/sklearncontainer"
)

testpath = sess.upload_data(
    path="attack_test.csv", bucket=bucket, key_prefix="sagemaker/sklearncontainer"
)

In [21]:
trainpath           # 확인

's3://sagemaker-ap-northeast-2-918934048065/sagemaker/sklearncontainer/attack_train.csv'

In [22]:
testpath            # 확인

's3://sagemaker-ap-northeast-2-918934048065/sagemaker/sklearncontainer/attack_test.csv'

## Writing a *Script Mode* script
The below script contains both training and inference functionality and can run both in SageMaker Training hardware or locally (desktop, SageMaker notebook, on prem, etc).

#### [reference]  
- script.py 작성하는 레퍼런스   
https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#preparing-the-scikit-learn-training-script


- (sklearn)TFidfVectorizer docs   
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html


- (sklearn)Support Vector Machines docs   
https://scikit-learn.org/stable/modules/svm.html#svc   


- (sklearn) make_pipeline  
https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html#sklearn.pipeline.make_pipeline




[ADD EXCEPTION(when len(jsonlist) !=4 -> 예외처리) 

In [23]:
# %%writefile script.py

# import argparse         # 명령행 인수를 파싱하기 위한 lib
# import joblib           # 모델을 저장하고 불러오기 위한 lib
# import os               # os <-경로등을 위한 라이브러리

# import numpy as np      # numpy,pandas 데이터 처리 및 조작를 위한  lib
# import pandas as pd
# import sklearn          # scikit-learn 라이브러리

# # from sklearn.ensemble import RandomForestRegressor

# from sklearn.feature_extraction.text import TfidfVectorizer     #
# from sklearn.svm import SVC
# from sklearn.pipeline import make_pipeline                      #여러 함수를 하나로 묶는 방법
# from sklearn.model_selection import GridSearchCV, train_test_split      

# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc

# import json         #json 형태로 데이터 전달해주기 위한 라이브러리

# # inference functions ---------------


# # Load the model
# def model_fn(model_dir):
#     """
#     Load the trained model from the specified model directory.

#     Parameters:
#     model_dir (str): Path to the directory where the model is saved.

#     Returns:
#     model: Loaded machine learning model.
#     """
#     model = joblib.load(os.path.join(model_dir, "model.joblib"))
#     return model



# # Modify input function to accept JSON list with four values
# def input_fn(request_body, request_content_type):
#     """
#     Parse the incoming JSON input.

#     Parameters:
#     request_body (str): Request body containing input data in JSON format (a list with four values).
#     request_content_type (str): Content type of the request.

#     Returns:
#     input_data: Parsed input data (a list of four values).
#     """
#     if request_content_type == "application/json":
#         try:
#             input_data = json.loads(request_body)
#             if len(input_data) != 4:
#                 raise ValueError("Invalid input. Expected a JSON list with four values.")
#             return input_data
#         except ValueError as e:
#             raise ValueError("Invalid input. Unable to parse JSON input: {}".format(e))
#     else:
#         raise ValueError("Invalid content type. Expected application/json.")

# # Modify prediction function to use the model with the extracted values
# def predict_fn(input_data, model):
#     """
#     Perform predictions using the loaded model.

#     Parameters:
#     input_data: Input data for prediction (a list of four values).
#     model: Loaded machine learning model.

#     Returns:
#     predictions: Predicted labels.
#     """
#     if model is None:
#         raise RuntimeError("Model has not been loaded.")
    
#     # Ensure that input_data is a list
#     if not isinstance(input_data, list) or len(input_data) != 4:
#         raise ValueError("Invalid input. Expected a list with four values.")
    
#     # Convert the input list to a numpy array and reshape it to match the input format
#     input_data = np.array(input_data).reshape(1, -1)
    
#     predictions = model.predict(input_data)
#     return predictions.tolist()

# # Modify output function to format the prediction results
# def output_fn(predictions, content_type):
#     """
#     Format the prediction results.

#     Parameters:
#     predictions: Predicted labels.
#     content_type (str): Content type for the output.

#     Returns:
#     output_data: Formatted output data.
#     """
#     if content_type == "application/json":
#         try:
#             output_data = json.dumps(predictions)
#             return output_data, content_type
#         except ValueError as e:
#             raise ValueError("Unable to serialize the prediction output to JSON: {}".format(e))
#     else:
#         raise ValueError("Invalid content type. Expected application/json.")





# if __name__ == "__main__":
#     print("[INFO] Extracting arguments")

#     parser = argparse.ArgumentParser()
                       
#     # Data, model, and output directories
#     parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
#     parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
#     parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
#     parser.add_argument("--train-file", type=str, default="attack_train.csv")
#     parser.add_argument("--test-file", type=str, default="attack_test.csv")   


#     args, _ = parser.parse_known_args()
    
#     print("SKLearn Version: ", sklearn.__version__)         #version check
#     print("Joblib Version: ", joblib.__version__)           #version check


#     print("[INFO] Reading data")
#     print()
#     train_df = pd.read_csv(os.path.join(args.train, args.train_file))
#     test_df = pd.read_csv(os.path.join(args.test, args.test_file))

#     print("building training and testing datasets")
#     X_train = train_df['pattern']
#     X_test = test_df['pattern']
#     y_train = train_df['label']
#     y_test = test_df['label']
    
#     # train
#     print("training model")

    
#     model = make_pipeline(TfidfVectorizer(input = 'content',
#                                           lowercase = True,
#                                           analyzer = 'char',
#                                           max_features = 1024,
#                                           ngram_range = (1,2),
#                                           decode_error="ignore"),
#                           SVC(C=10, kernel ='rbf'))



#     # 모델 저장
#     model.fit(X_train, y_train)
#     # persist model
#     path = os.path.join(args.model_dir, "model.joblib")
#     joblib.dump(model, path)
#     print("model persisted at " + path)
#     # print(args.min_samples_leaf)
    
    
#     y_pred_test = model.predict(X_test)
#     test_acc = accuracy_score(y_test,y_pred_test)
#     test_rep = classification_report(y_test,y_pred_test)

#     print()
#     print("---- METRICS RESULTS FOR TESTING DATA ----")
#     print()
#     print("Total Rows are: ", X_test.shape[0])
#     print('[TESTING] Model Accuracy is: ', test_acc)
#     print('[TESTING] Testing Report: ')
#     print(test_rep)

In [24]:
%%writefile script.py

import argparse         # 명령행 인수를 파싱하기 위한 lib
import joblib           # 모델을 저장하고 불러오기 위한 lib
import os               # os <-경로등을 위한 라이브러리

import numpy as np      # numpy,pandas 데이터 처리 및 조작를 위한  lib
import pandas as pd
import sklearn          # scikit-learn 라이브러리

# from sklearn.ensemble import RandomForestRegressor

from sklearn.feature_extraction.text import TfidfVectorizer     #
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline                      #여러 함수를 하나로 묶는 방법
from sklearn.model_selection import GridSearchCV, train_test_split      

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc

import json         #json 형태로 데이터 전달해주기 위한 라이브러리

# inference functions ---------------


# Load the model
def model_fn(model_dir):
    """
    Load the trained model from the specified model directory.

    Parameters:
    model_dir (str): Path to the directory where the model is saved.

    Returns:
    model: Loaded machine learning model.
    """
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    return model

# Input function to parse the incoming JSON input
def input_fn(request_body, request_content_type):
    """
    Parse the incoming JSON input.

    Parameters:
    request_body (str): Request body containing input data in JSON format.
    request_content_type (str): Content type of the request.

    Returns:
    input_data: Parsed input data.
    """
    if request_content_type == "application/json":
        try:
            input_data = json.loads(request_body)
            return input_data
        except ValueError as e:
            raise ValueError("Invalid input. Unable to parse JSON input: {}".format(e))
    else:
        raise ValueError("Invalid content type. Expected application/json.")

        
        
# Prediction function
def predict_fn(input_data, model):
    """
    Perform predictions using the loaded model.

    Parameters:
    input_data: Input data for prediction.
    model: Loaded machine learning model.

    Returns:
    predictions: Predicted labels.
    """
    if model is None:
        raise RuntimeError("Model has not been loaded.")

    predictions = model.predict(input_data)
    return predictions.tolist()



# Output function to format the prediction results
def output_fn(predictions, content_type):
    """
    Format the prediction results.

    Parameters:
    predictions: Predicted labels.
    content_type (str): Content type for the output.

    Returns:
    output_data: Formatted output data.
    """
    if content_type == "application/json":
        try:
            output_data = json.dumps(predictions)
            return output_data, content_type
        except ValueError as e:
            raise ValueError("Unable to serialize the prediction output to JSON: {}".format(e))
    else:
        raise ValueError("Invalid content type. Expected application/json.")







if __name__ == "__main__":
    print("[INFO] Extracting arguments")

    parser = argparse.ArgumentParser()
                       
    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="attack_train.csv")
    parser.add_argument("--test-file", type=str, default="attack_test.csv")   


    args, _ = parser.parse_known_args()
    
    print("SKLearn Version: ", sklearn.__version__)         #version check
    print("Joblib Version: ", joblib.__version__)           #version check


    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    print("building training and testing datasets")
    X_train = train_df['pattern']
    X_test = test_df['pattern']
    y_train = train_df['label']
    y_test = test_df['label']
    
    # train
    print("training model")

    
    model = make_pipeline(TfidfVectorizer(input = 'content',
                                          lowercase = True,
                                          analyzer = 'char',
                                          max_features = 1024,
                                          ngram_range = (1,2),
                                          decode_error="ignore"),
                          SVC(C=10, kernel ='rbf'))



    # 모델 저장
    model.fit(X_train, y_train)
    # persist model
    path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, path)
    print("model persisted at " + path)
    # print(args.min_samples_leaf)
    
    
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)

Overwriting script.py


## Local training
Script arguments allows us to remove from the script any SageMaker-specific configuration, and run locally

In [25]:
# ! python script.py --C 10 \
#                    --model-dir ./ \
#                    --train ./ \
#                    --test ./ \


! python script.py --model-dir ./ \
                   --train ./ \
                   --test ./ \

[INFO] Extracting arguments
SKLearn Version:  1.0.1
Joblib Version:  1.3.2
[INFO] Reading data

building training and testing datasets
training model
model persisted at ./model.joblib

---- METRICS RESULTS FOR TESTING DATA ----

Total Rows are:  3500
[TESTING] Model Accuracy is:  0.9788571428571429
[TESTING] Testing Report: 
               precision    recall  f1-score   support

accesscontrol       0.99      0.97      0.98       226
     download       0.93      1.00      0.96        13
     includei       1.00      1.00      1.00        11
 logical_sqli       0.86      1.00      0.92         6
    shellcode       0.94      0.99      0.97       157
         sqli       0.96      0.88      0.92        50
       upload       0.68      1.00      0.81        99
     urlregex       0.98      1.00      0.99       428
        valid       1.00      1.00      1.00      2404
    webattack       1.00      0.20      0.34        59
          xss       0.95      0.85      0.90        47

     accura

## SageMaker Training

### Launching a training job with the Python SDK   

[reference]    
https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html   

In [26]:
# Import necessary module from the SageMaker Python SDK
from sagemaker.sklearn.estimator import SKLearn

# Specify the scikit-learn framework version to use
FRAMEWORK_VERSION = "0.23-1"

# Create an SKLearn estimator with the specified configuration
sklearn_estimator = SKLearn(
    entry_point="script.py",         # The entry point script for training
    role=get_execution_role(),       # SageMaker IAM role for executing training jobs
    instance_count=1,                # Number of EC2 instances to use for training
    instance_type="ml.m5.large",     # Type of EC2 instance for training
    framework_version=FRAMEWORK_VERSION,  # The version of scikit-learn framework to use
    base_job_name="svc-custom-sklearn"  # Base name for the training job
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [27]:
# launch training job, with asynchronous call
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)

INFO:sagemaker:Creating training-job with name: svc-custom-sklearn-2023-11-21-02-23-50-773


Using provided s3_resource
2023-11-21 02:23:51 Starting - Starting the training job...
2023-11-21 02:24:07 Starting - Preparing the instances for training......
2023-11-21 02:25:05 Downloading - Downloading input data...
2023-11-21 02:25:40 Training - Downloading the training image...
2023-11-21 02:26:10 Training - Training image download completed. Training in progress.2023-11-21 02:26:13,212 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-11-21 02:26:13,215 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-11-21 02:26:13,257 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-11-21 02:26:13,419 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-11-21 02:26:13,431 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-11-21 02:26:13,443 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus

## Store Model Artifacts(model.tar.gz) into the S3 Bucket

In [28]:
# Wait for the completion of the latest training job and retrieve logs
sklearn_estimator.latest_training_job.wait(logs="None")

# Retrieve the S3 path of the model artifact from the completed training job
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

# Print the S3 path where the model artifact is stored
print("Model artifact persisted at " + artifact)


2023-11-21 02:26:41 Starting - Preparing the instances for training
2023-11-21 02:26:41 Downloading - Downloading input data
2023-11-21 02:26:41 Training - Training image download completed. Training in progress.
2023-11-21 02:26:41 Uploading - Uploading generated training model
2023-11-21 02:26:41 Completed - Training job completed
Model artifact persisted at s3://sagemaker-ap-northeast-2-918934048065/svc-custom-sklearn-2023-11-21-02-23-50-773/output/model.tar.gz


### Deploy Sagemaker Endpoint(API) for trained model

In [29]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

# Define a unique model name based on the current timestamp
# model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_name = "pyree"

# Create a SageMaker SKLearnModel
model = SKLearnModel(
    name=model_name,             # Name for the SageMaker model
    model_data=artifact,         # S3 path of the model artifact obtained earlier
    role=get_execution_role(),   # IAM role for accessing resources
    entry_point="script.py",     # Entry point script for inference
    framework_version=FRAMEWORK_VERSION  # Scikit-learn framework version
)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [30]:
# Define a unique endpoint name based on the current timestamp

endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
# endpoint_name = "pyree"

# Print the endpoint name
print("EndpointName={}".format(endpoint_name))

# Deploy the model to a SageMaker endpoint
predictor = model.deploy(
    initial_instance_count=1,     # Number of initial instances for the endpoint
    instance_type="ml.c5.xlarge", # Type of EC2 instance for the endpoint
    endpoint_name=endpoint_name,  # Name of the endpoint to be created
    
    # Optional: endpoint configuration (you can uncomment and use if needed)
    # endpoint_config=my_endpoint_config,
)

EndpointName=Custom-sklearn-model-2023-11-21-02-27-08
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


INFO:sagemaker:Creating model with name: pyree
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model-2023-11-21-02-27-08
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model-2023-11-21-02-27-08


-------*

UnexpectedStatusException: Error hosting endpoint Custom-sklearn-model-2023-11-21-02-27-08: Failed. Reason: Failed to download model data from URL "s3://sagemaker-ap-northeast-2-918934048065/svc-custom-sklearn-2023-10-21-15-42-21-683/output/model.tar.gz". Please ensure that there is an object located at the URL and that the role passed to CreateModel has permissions to download the object..

### Alternative : invoke with boto3

[reference]  
https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints-invoke.html   


### Content-Type : application/json

In [ ]:
import boto3
import json

runtime = boto3.client("sagemaker-runtime")
endpoint_name = predictor.endpoint
print(endpoint_name)

# Print the prediction result
input_data = ["param=mv a b", "1=1,kwqdjqwkd", "2faf2430c9a04a39", "/erpuupdate2/erpuupdateservice.asmx"]
# Convert the input data to JSON format
input_json = json.dumps(input_data)

# Send a request to the endpoint for prediction
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=input_json,
)

# Parse the response
result = json.loads(response["Body"].read().decode())

# Print the prediction result
print("Prediction result:", result)


#### Don't forget to delete the endpoint!


- Amazon SageMaker 요금   
https://aws.amazon.com/ko/sagemaker/pricing/

In [ ]:
# sm_boto3.delete_endpoint(EndpointName=endpoint_name)